In [3]:
"""
Construct a De Bruijn graph (k=5) from sequencing reads
and output the graph in GFA format (nodes + edges only).

Author: Yifei Wang
Date: Dec 18, 2025
"""

from collections import defaultdict

# -----------------------------
# Input reads
# -----------------------------
reads = [
    "TTGCAT", "TGAACT", "ATTGCA", "TGCATT",
    "CATTTG", "TGGTAA", "ATTTGA", "AACTAG",
    "TAGCAT", "TAGGTA", "GCATTT", "TTGAAC",
    "TAGGTA", "GGTAGC", "GTAGCA", "AGCATT"
]

k = 5
k1 = k - 1          # node length (4)
overlap = k - 2     # overlap length for GFA (3)


# -----------------------------
# Step 1: extract all 5-mers
# -----------------------------
kmer_count = defaultdict(int)

for read in reads:
    for i in range(len(read) - k + 1):
        kmer = read[i:i + k]
        kmer_count[kmer] += 1


# -----------------------------
# Step 2: collect nodes (4-mers)
# -----------------------------
nodes = set()

for kmer in kmer_count:
    prefix = kmer[:k1]
    suffix = kmer[1:]
    nodes.add(prefix)
    nodes.add(suffix)


# -----------------------------
# Step 3: assign node IDs
# -----------------------------
node_ids = {}
for i, node in enumerate(sorted(nodes)):
    node_ids[node] = f"utg{i}"


# -----------------------------
# Step 4: write GFA file
# -----------------------------
with open("debruijn_graph.gfa", "w") as gfa:
    # Header
    gfa.write("H\tVN:Z:1.0\n")

    # S-lines (nodes)
    for node, node_id in node_ids.items():
        gfa.write(f"S\t{node_id}\t{node}\tLN:i:{len(node)}\n")

    # L-lines (edges)
    for kmer, count in kmer_count.items():
        prefix = kmer[:k1]
        suffix = kmer[1:]

        from_id = node_ids[prefix]
        to_id = node_ids[suffix]

        # One L-line per unique k-mer
        gfa.write(
            f"L\t{from_id}\t+\t{to_id}\t+\t{overlap}M\n"
        )

print("GFA file written to debruijn_graph.gfa")

# ------------------------------------------------
# Step 5: Print results (human-readable)
# ------------------------------------------------
print("\n========== De Bruijn Graph Summary ==========\n")

print("NODES (4-mers):")
for node in sorted(node_ids):
    print(f"{node_ids[node]}: {node}")

print("\nEDGES (5-mers):")
for i, (kmer, count) in enumerate(sorted(kmer_count.items()), 1):
    prefix = kmer[:k1]
    suffix = kmer[1:]

    print(
        f"E{i}: {node_ids[prefix]} -> {node_ids[suffix]} | "
        f"Sequence: {kmer} | Count: {count}"
    )

print("\n============================================\n")







GFA file written to debruijn_graph.gfa

========== De Bruijn Graph Summary ==========

NODES (4-mers):
utg0: AACT
utg1: ACTA
utg2: AGCA
utg3: AGGT
utg4: ATTG
utg5: ATTT
utg6: CATT
utg7: CTAG
utg8: GAAC
utg9: GCAT
utg10: GGTA
utg11: GTAA
utg12: GTAG
utg13: TAGC
utg14: TAGG
utg15: TGAA
utg16: TGCA
utg17: TGGT
utg18: TTGA
utg19: TTGC
utg20: TTTG

EDGES (5-mers):
E1: utg0 -> utg1 | Sequence: AACTA | Count: 1
E2: utg1 -> utg7 | Sequence: ACTAG | Count: 1
E3: utg2 -> utg9 | Sequence: AGCAT | Count: 2
E4: utg3 -> utg10 | Sequence: AGGTA | Count: 2
E5: utg4 -> utg19 | Sequence: ATTGC | Count: 1
E6: utg5 -> utg20 | Sequence: ATTTG | Count: 2
E7: utg6 -> utg5 | Sequence: CATTT | Count: 2
E8: utg8 -> utg0 | Sequence: GAACT | Count: 1
E9: utg9 -> utg6 | Sequence: GCATT | Count: 3
E10: utg10 -> utg11 | Sequence: GGTAA | Count: 1
E11: utg10 -> utg12 | Sequence: GGTAG | Count: 1
E12: utg12 -> utg13 | Sequence: GTAGC | Count: 2
E13: utg13 -> utg2 | Sequence: TAGCA | Count: 2
E14: utg14 -> utg3 | Seque